In [26]:
%matplotlib inline

import networkx as nx
import matplotlib.pyplot as plt
import random

def get_graph(filename, b_enforce_random = False):
    
    if b_enforce_random:
        random_start = 0
        random_end = 5000
        random_target = random_start + (random_end - random_start) / 2
    
    f = open(data_file, 'r')
    g = nx.DiGraph()
    
    for line in f:
    
        year, from_case, to_case = line[:-1].split(',')
        
        if (b_enforce_random == False) or (random_target == random.randrange(random_start, random_end)):
            g.add_edge(from_case, to_case, attr_dict={'year': year})    
        
    f.close()
    
    return g
    

def display_graph(graph):
    
    print graph.nodes()


def topological_sort(graph):
    """
    Sorts graph by year.
    Argument: graph
    Returns: a list of nodes of the graph sorted by dependencies
    """
    
    return nx.topological_sort(graph)


In [27]:
data_file = 'Data/graph_mini.csv'

g = get_graph(data_file, True)

print 'Graph node order:'
display_graph(g)

Graph node order:
['X9UDUC', 'X4ABG7', 'XFLI1B', 'X4A71K', 'X3FTA6', 'XFIFOJ', 'XFKNAV', 'X1VT5V', 'X44I6Q', 'XOKKGBQNB5G0', 'X4924A', 'X4AMEI', 'X44MD6', 'XFL3KF', 'X4A8AC', 'X48FGJ', 'XFH3IE', 'XFKJ7G', 'X3D2H5', 'X49Q52', 'X3E4AE', 'XFPU92', 'XFK96A', 'X48SH0', 'X3OETI', 'X40LDO', 'XFIHGQ', 'XFKIBN', 'X443VV', 'X44CGS', 'X48E5D', 'X3Q17K', 'X47MMG', 'X9VBKA', 'X41AEN', 'X498JE', 'X46HDD', 'X4AIDH', 'XFL8M4', 'X44MRU', 'X449P9', 'X44J89', 'X3F940', 'XASDBBQNB5G0', 'XFK962', 'X3ID60', 'X48A0K', 'XFHRQB', 'X43BTP', 'XFLND0', 'XFPHTS', 'XFIF7A', 'X3G60H', 'XIDP3F', 'X4LPSB', 'X41C6G', 'XA6Q9L', 'X44EKB', 'XFPMFJ', 'X48PDJ', 'X44IUF', 'X3L3VB', 'X481JN', 'X48C5H', 'X3OH48', 'X407BM', 'XOKHJRQNB5G0', 'XFKMD0', 'X3H3BD', 'X3I889', 'X3D6DS', 'X47RKG', 'XFLUPC', 'X46CQ7', 'X47NAF', 'X3BB3O', 'X3ITKK', 'XFJ8KG', 'X3U080', 'XFKPJL', 'XB0LGNQNB5G0', 'X47REL', 'X43DNI', 'XFKDHJ', 'X3E499', 'X2D6JN', 'X3IGUN', 'XFLFGN', 'XFLUI1', 'X3GQI3', 'X4149T', 'XFLUFP', 'XFKMLD', 'XFL1E1', 'XBBCO7QNB5G0', '

In [33]:
sorted_list = topological_sort(g)

print 'Graph node order AFTER sorting:'
print sorted_list





Graph node order AFTER sorting:
['XFLPCV', 'XFJCPP', 'X9VB6I', 'XFLVC1', 'XFHUMM', 'XFL6V6', 'X4AIOU', 'X3VTBV', 'X3IETP', 'X44PA3', 'XIFDVB', 'X3JI3M', 'X3V6AF', 'X47L90', 'X41QEV', 'X3H36K', 'XFL531', 'XFKLLA', 'X4628J', 'X3BINH', 'X3IKCK', 'XFLP5C', 'XFII3V', 'XFIHS1', 'XFJ64Q', 'X44D6J', 'XFITKR', 'X3OGS7', 'X3CEMC', 'XFLBCI', 'XFL9C9', 'X48ICR', 'X3D6HQ', 'XAREE7QNB5G0', 'X447JG', 'X9VD64', 'X41B29', 'XFL4Q1', 'XFLI59', 'XFLHJS', 'XG19RK', 'XFKUTM', 'XFHPU2', 'X3BIKR', 'X44PEE', 'X41HOI', 'XFJ8B8', 'XFL1G6', 'XIEO57', 'XBBAVRQNB5G0', 'XFL1N5', 'XFLQ4D', 'X3Q13U', 'X3ERII', 'X9VC72', 'X3JEUM', 'X3IM9N', 'X3C9RS', 'XFL9JB', 'X48UOV', 'X44P5J', 'X48STS', 'X46AAD', 'X447CL', 'X41F3S', 'XFL5T0', 'XFLCFJ', 'X488U8', 'X40SEV', 'XFLB4G', 'XFM2BO', 'X9VBMT', 'X3D2IF', 'X41N61', 'X41GBH', 'X48NBP', 'X418GM', 'X9VF8N', 'X3C9TK', 'XFHUS8', 'XFLCOL', 'XFKINK', 'X3D6JN', 'XFHK7A', 'XFKHJ4', 'X3H4HT', 'X3BCJD', 'XIGA8V', 'X48P1R', 'X48MV2', 'XFLMT5', 'XFJ0IP', 'XFKKVN', 'X47QB2', 'X3BH63', '

In [34]:
# Accessing children of nodes in sorted list

for node in sorted_list:
    
    print nx.edges(g, node)

[('XFLPCV', 'XOKHJRQNB5G0')]
[('XFJCPP', 'XFKLLA')]
[('X9VB6I', 'XFKHJ4')]
[('XFLVC1', 'XFPMFJ')]
[('XFHUMM', 'XFL6V6')]
[]
[('X4AIOU', 'X9VC72')]
[('X3VTBV', 'XBBCO7QNB5G0')]
[('X3IETP', 'X3L3VB')]
[('X44PA3', 'X48FGJ')]
[('XIFDVB', 'XIEO57')]
[('X3JI3M', 'X3ITKK')]
[('X3V6AF', 'X9VBKA')]
[('X47L90', 'X9UDUC')]
[('X41QEV', 'X3D6DS')]
[('X3H36K', 'X449P9')]
[('XFL531', 'XFL4Q1')]
[]
[('X4628J', 'X3IM9N')]
[('X3BINH', 'X3JEUM')]
[('X3IKCK', 'XFLP5C')]
[]
[('XFII3V', 'XFLCFJ')]
[('XFIHS1', 'XFJ64Q')]
[]
[('X44D6J', 'X3BB3O')]
[('XFITKR', 'XFL9JB')]
[('X3OGS7', 'XFLFGN')]
[('X3CEMC', 'XFLBCI')]
[]
[('XFL9C9', 'XFKUTM')]
[('X48ICR', 'X3C9RS')]
[('X3D6HQ', 'X46AAD')]
[('XAREE7QNB5G0', 'XFKNAV')]
[('X447JG', 'X9VD64')]
[]
[('X41B29', 'X3H3BD')]
[]
[('XFLI59', 'XFLHJS')]
[]
[('XG19RK', 'XFIF7A')]
[]
[('XFHPU2', 'XOKP4NQNB5G0')]
[('X3BIKR', 'X467U2')]
[('X44PEE', 'X41HOI')]
[]
[('XFJ8B8', 'XFKMD0')]
[('XFL1G6', 'XFK215')]
[]
[('XBBAVRQNB5G0', 'XFL1N5')]
[]
[('XFLQ4D', 'XFKPJL')]
[('X3Q13U', 'X